In [37]:
%load_ext autoreload
%autoreload 2


import os
import time
import copy
import json
import openai
from dotenv import load_dotenv

from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from method.ours.utils import get_application_context

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA'
GRAPH_EMBEDDING_METHOD = 'NODE2VEC'

URL = 'https://www.aa.com/homePage.do'
# 'https://www.jetblue.com/'
# 'https://www.united.com/en/us'
# 'https://www.aircanada.com/ca/en/aco/home.html'


def get_to_form(driver):
    try:
        driver.get(URL)
        
        '''
        # AC - My Bookings
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
        ).click()
        '''
        
        '''
        # AC - Multi-city
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
        ).click()
        '''
        
    except:
        print('timeout')


def find_form():
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/MAIN[1]/DIV[1]/DIV[2]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]'
        ))
    )


def find_button():
    return None

In [4]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [5]:
html = driver.find_element(By.TAG_NAME, 'html').get_attribute('outerHTML')
form = find_form()
form_xpath = get_xpath(driver, form)

inputs = form.find_elements(
    By.XPATH,
    f'{form_xpath}//input | {form_xpath}//textarea | {form_xpath}//select'
)

inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

In [ ]:
app_context = get_application_context(driver).replace('\n', ' ')

In [40]:
input_widget_context = ''

for i in inputs:
    if i.tag_name != 'input':
        continue
    
    input_id = i.get_attribute('id')
    
    try:
        label = form.find_element(By.XPATH, f'//*[@for="{input_id}"]').text.replace('\n', ' ')
        input_widget_context += f'This input is about {label}. {label} is [MASK]. '
    except:
        pass

# print(input_widget_context)

In [44]:
FORM_FUNC = 'Book'

prompt = f'''
The app is {app_context}, in its {FORM_FUNC} form. {input_widget_context}
'''.strip()

In [45]:
prompt

'The app is Title: American Airlines - Airline tickets and low fares at aa.com Description: Book low fares to destinations around the world and find the latest deals on airline tickets, hotels, car rentals and vacations at aa.com. As an AAdantage member you earn miles on every trip and everyday spend., in its Book form. This input is about Search flight Flight. Search flight Flight is [MASK]. This input is about Search vacation packages Vacation packages. Search vacation packages Vacation packages is [MASK]. This input is about Search flights round trip Round trip. Search flights round trip Round trip is [MASK]. This input is about Search flights one way One way. Search flights one way One way is [MASK]. This input is about Redeem miles. Redeem miles is [MASK]. This input is about From. From is [MASK]. This input is about To. To is [MASK]. This input is about Depart (date format mm/dd/yyyy). Depart (date format mm/dd/yyyy) is [MASK]. This input is about Return (date format mm/dd/yyyy).

In [46]:
# This code is for v1 of the openai package: pypi.org/project/openai
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "system",
      "content": "You are required to generate values for a given input or list of inputs given in a text. Please create an array of values, the first element is a passing value for the inputs, and the rest are failing values. Please note that each [MASK] is showing a different input. Give the values for different inputs in arrays, like the following format:\n\n[\n[First input, second input, ...],\n[First input, second input, ...],\n...\n]"
    },
    {
      "role": "user",
      "content": prompt
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [55]:
values = json.loads(response.choices[0].message.content)

passing = values[0]
failing = values[1:]

print("Passing:", passing)
print()

for f in failing:
    print("Failing:", f)
    print()

Passing: ['On', 'Unavailable', 'Return', 'Single', '20,000', 'Seattle', 'Houston', '09/05/2022', '09/15/2022', 'New York', 'Chicago', '11/11/2022', '11/20/2022']

Failing: ['Off', 'No results found', 'One way', 'Round Trip', '0', 'Houston', 'Same location', '32/15/2022', '15/09/2022', 'Seattle', 'Same city', '15/33/2022', '22/09/2022']

Failing: ['Not Available', 'Network error', 'Wrong choice', 'Cannot select', 'Less than need', 'No city name', 'Transportation method', '2022/09/05', '2022/15/09', 'Transport', 'No destination', '05/2022/09', '2022/09/15']

Failing: ['Server error', 'Slow response', 'Already selected', 'Not allowed', 'Greater than available', 'Unknown location', 'No city', '200522', '152022', 'Unknown', 'Same as from', '20220915', '20220922']

